<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2021notebooks/2021_1223BERT_onomatopea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- date: 2021_1223
- filename: 2021_1223BERT_onomatopea.ipynb
- source: https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2021notebooks/2021_1223BERT_onomatopea.ipynb
- author: 浅川伸一

# 日本語オノマトペの BERT マスク化言語モデルによる出力例

In [ ]:
import os
import sys
import numpy as np
from termcolor import colored

# 必要なライブラリのインストール
import platform
isColab = True if platform.system() == 'Linux' else False
if isColab:
    !pip install transformers > /dev/null 2>&1 

    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                     shell=True).communicate()[0]).decode('utf-8')

    !pip install 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download > /dev/null 2>&1
    !pip install ipadic > /dev/null 2>&1

In [ ]:
# ここで `日本語オノマトペ辞典4500より.xls` と `original.csv` を指定してアップロードする
# 両ファイルは共に著作権保護対象の内容であり，公にできません。
from google.colab import files
uploaded = files.upload()  

In [ ]:
# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータの読み込み
import pandas as pd
import unicodedata
#import jaconv

onomatopea_excel = '日本語オノマトペ辞典4500より.xls'  # オリジナルファイル名，次行は勝手に rename したファイル名
#onomatopea_excel = '2021-0325日本語オノマトペ辞典4500より.xls'
onmtp2761 = pd.read_excel(onomatopea_excel, sheet_name='2761語')

onomatopea = list(sorted(set([o for o in onmtp2761['オノマトペ']])))
print(f'データファイル名: {onomatopea_excel}\n',
      f'オノマトペ単語総数: {len(onomatopea)}')

# 近藤先生 (2021年12月22日） から送っていただいた，オノマトペ文章データを読み込む
original = {}
n = 0
with open('original.csv', 'r', encoding='utf8') as f:
    s = f.read()
    for s_ in s.split('\n'):
        if n == 0:
            n += 1
            continue
        idx, sent = s_.split(',')
        
        # Mac と Windows との unicode 符号化の差分を吸収する
        sent = ''.join(unicodedata.normalize('NFKC',x) for x in sent)
        original[int(idx)] = {'sent':sent}

print(f'{len(original)} has been read')

In [ ]:
import torch
from transformers import BertConfig
from transformers import BertModel
from transformers import BertForPreTraining
from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM

model_ja_name = 'cl-tohoku/bert-base-japanese' 
model = BertModel.from_pretrained(model_ja_name)
config = BertConfig.from_pretrained(model_ja_name)

# トークナイザ を 2 つ使ってみる
tknz1 = BertJapaneseTokenizer.from_pretrained(model_ja_name)
tknz2 = BertJapaneseTokenizer.from_pretrained(model_ja_name, do_subword_tokenize=False)

#print(len(tknz.vocab), len(onomatopea))
n_added = tknz1.add_tokens(onomatopea)    # 東北大学 BERT に登録されていないオノマトペを加える。
tknz1.ids_to_tokens.update()

n_added_ = tknz2.add_tokens(onomatopea)  # 東北大学 BERT に登録されていないオノマトペを加える。
tknz2.ids_to_tokens.update()
model.resize_token_embeddings(len(tknz1))

print(len(tknz1), len(tknz1.vocab), tknz1.vocab_size)
print(len(tknz2), len(tknz2.vocab), tknz2.vocab_size)

print(tknz1.convert_tokens_to_ids(onomatopea[-10:]))
print(tknz1.convert_ids_to_tokens(tknz1.convert_tokens_to_ids(onomatopea[-13:])))

print(tknz2.convert_tokens_to_ids(onomatopea[-10:]))
print(tknz2.convert_ids_to_tokens(tknz2.convert_tokens_to_ids(onomatopea[-13:])))


In [ ]:
import re
# ランダムサンプリングしてデータを印字して確認
for _ in range(5):
    N = np.random.randint(low=0, high=len(original))
    sent0 = original[N]['sent']
    sent1 = re.sub(r'\(.*\)','',sent) # original に含まれる `(と)` のような表現を削除する
    
    print(colored(sent0, attrs=['bold']))  # 送っていただいた元の文
    print(colored('分かち書き','blue'), tknz1.tokenize(sent0)) # その分かち書き
    print(colored('分かち書き','green'), tknz2.tokenize(sent0)) # その分かち書き
    print(colored('ID 化', 'blue'), tknz1.encode(sent0))   # 分かち書き結果の単語 ID 化
    print(colored('ID 化', 'green'), tknz2.encode(sent0))   # 分かち書き結果の単語 ID 化
    if sent0 != sent1:
        print(colored(sent1, attrs=['bold']))   # (と) を取り去った文
        print(colored('分かち書き','blue'), tknz1.tokenize(sent1)) # その分かち書き
        print(colored('分かち書き','green'), tknz2.tokenize(sent1)) # その分かち書き
        print(colored('ID 化', 'blue'), tknz1.encode(sent1))   # 分かち書き結果の単語 ID 化
        print(colored('ID 化', 'green'), tknz2.encode(sent1))   # 分かち書き結果の単語 ID 化


# MeCab で単語分割が行われて、MeCab が単語として認識しても、その単語が語鎮リスト vocab.txt に登録されていない場合は
# subword である WordPiece が起動され、その単語が適当に分割されます。そのように分割された単語には '##' が単語の前に付与されます。
# また、未知語の場合もWordPieceが起動され、同様に分割されます。
print(colored(f'\ntknz.all_special_ids:{tknz1.all_special_ids}',attrs=['bold']))  #  [1, 3, 0, 2, 4]
print(colored(f'tknz.all_special_tokens:{tknz1.all_special_tokens}', attrs=['bold']))  #  ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [ ]:
#print(dir(tknz))  # トークナイザの内部を調べてみましょう

In [ ]:
from termcolor import colored
# トークナイザの属性値を表示して理解を深める
for k in ['sep_token', 'sep_token_id', 'slow_tokenizer_class', 
          'unk_token', 'unk_token_id', 'verbose', #'vocab', 
          'vocab_files_names', 'vocab_size', 'word_tokenizer', 'word_tokenizer_type',
          'special_tokens_map', 'special_tokens_map_extended', 
          'subword_tokenizer', 'subword_tokenizer_type',
          'all_special_ids', 'all_special_tokens', 'unk_token', 'unk_token_id', 
          'special_tokens_map', 'special_tokens_map_extended', 
          'subword_tokenizer', 'subword_tokenizer_type', 
          'tokenize',
          'max_model_input_sizes', 'mecab_kwargs', 'model_input_names', 'model_max_length']:
    print(colored(f'{k}','yellow'), colored(f'{getattr(tknz1,k)}','green'))

In [ ]:
# オノマトペにマッチする部分を検索し，[MASK] で置き換える
masked_onmtp = {}
for N in original:
    sent = original[N]['sent']
            
    for i, o in enumerate(onomatopea):
        if sent.find(o) != -1:
            target_id = i
            o_ = o  # 最後にマッチしたオノマトペだけ取り出す
            
    if o_ not in masked_onmtp:
        masked_onmtp[o_] = [sent.replace(o_,'[MASK]')]
    else:
        masked_onmtp[o_].append(sent.replace(o_,'[MASK]'))

In [ ]:
from transformers import BertForMaskedLM
masked_lm1 = BertForMaskedLM.from_pretrained(model_ja_name) #'cl-tohoku/bert-base-japanese')
masked_lm2 = BertForMaskedLM.from_pretrained(model_ja_name) #'cl-tohoku/bert-base-japanese')

tknz1.add_tokens(onomatopea)
n_added = tknz1.add_tokens(onomatopea)    # 東北大学 BERT に登録されていないオノマトペを加える。
tknz1.ids_to_tokens.update()
masked_lm1.resize_token_embeddings(len(tknz1))

tknz2.add_tokens(onomatopea)
n_added_ = tknz2.add_tokens(onomatopea)  # 東北大学 BERT に登録されていないオノマトペを加える。
tknz2.ids_to_tokens.update()
masked_lm2.resize_token_embeddings(len(tknz2))

n_limit = 3   # デバッグのため総出力回数を 3 にしていますが，すべての出力を得るためには，n_limit の値を len(masked_onmtp) にしてください
for i, s in enumerate(masked_onmtp):
    print(colored(f'{i} {s}',attrs=['bold']), masked_onmtp[s])
    for s_ in masked_onmtp[s]:
        print(tknz1.tokenize(s_), end="")
        print(colored(tknz1(s_)['input_ids'], 'green'), end="") # with sentencepiece
        x = torch.LongTensor(tknz1(s_)['input_ids']).unsqueeze(0)
        a = masked_lm1(x)
        print(a[0].shape)

        print(tknz2.tokenize(s_), end="")
        print(colored(tknz2(s_)['input_ids'], 'cyan'), end="") # without sentencepiece
        x = torch.LongTensor(tknz2(s_)['input_ids']).unsqueeze(0)
        a = masked_lm2(x)
        print(a[0].shape)

        idxs = tknz2(s_)['input_ids']
        maskpos = idxs.index(tknz2.mask_token_id)
        top_n = 5
        b = torch.topk(a.to_tuple()[0][0][maskpos], k=top_n)
        b_idx = b[1].detach().numpy()
        b_wrd = list(tknz2.convert_ids_to_tokens(b_idx))
        for k in range(top_n):
            for j, idx in enumerate(idxs):
                if j != maskpos:
                    print(colored(tknz2.convert_ids_to_tokens(idx),'blue'), end=" ")
                else:
                    print(colored(b_wrd[k],'red', attrs=['bold']),end=" ")
                    
            print()
    if i > n_limit:
        break

In [ ]:
def print_message(a, s_, tokenizer=tknz1, top_n=5):
    idxs = tokenizer(s_)['input_ids']
    maskpos = idxs.index(tokenizer.mask_token_id)
    b = torch.topk(a.to_tuple()[0][0][maskpos], k=top_n)
    b_idx = b[1].detach().numpy()
    b_wrd = list(tokenizer.convert_ids_to_tokens(b_idx))
    for k in range(top_n):
        print(k, end=":")
        for j, idx in enumerate(idxs):
            if j != maskpos:
                print(colored(tokenizer.convert_ids_to_tokens(idx),'grey'), end=" ")
            else:
                print(colored(b_wrd[k],'red', attrs=['bold']),end=" ")
        print()

n_limit = 3  
# デバッグのため総出力回数を 3 にしていますが，すべての出力を得るためには，n_limit の値を len(masked_onmtp) 
# にするか，または直下行を有効にして，次々行をコメントアウトしてください
#for i, s in enumerate(masked_onmtp):
for i in range(5):
    N = np.random.choice(len(masked_onmtp))
    N = 49
    N = 4
    ono = list(masked_onmtp.keys())[N]
    s = masked_onmtp[ono]
    print(colored(f'{i} N:{N} オノマトペ:{ono} {s}', 'blue', attrs=['bold']))
    for s_ in s:
        print(tknz1.tokenize(s_), end="")
        print(colored(tknz1(s_)['input_ids'], 'green'), end="") # with sentencepiece
        x = torch.LongTensor(tknz1(s_)['input_ids']).unsqueeze(0)
        a = masked_lm1(x)
        print(a[0].shape)
        
        print('=' * 7, 'with wordpice')
        print_message(a, s_, tokenizer=tknz1)
        print('+' * 7, 'withoout wordpeice')
        print_message(a, s_, tokenizer=tknz2)


In [ ]:
tknz1.save_vocabulary('vocab_saved.txt')  # 後に利用可能なように，語彙辞書をテキストファイルとして書き出す

# 結果の確認
!head vocab_saved.txt
!tail vocab_saved.txt
#help(tknz.save_vocabulary)